In [1]:
import sys 
sys.path.append('/Users/justinvhuang/Desktop/CSE-6242-Group-Project')

In [2]:
import guidance

from guidance import gen, models
from tqdm import tqdm

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from tqdm import tqdm
from utils.textpreprocessing import TextPreprocessor

In [3]:
textprepo = TextPreprocessor()

In [5]:
encode_kwargs = {"normalize_embeddings": True}
embedding_function = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={"device": "cpu"},
    encode_kwargs=encode_kwargs,
)

In [7]:
new_db = FAISS.load_local("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/vector_database_creation/faiss_anime_index_v2", embedding_function)

In [8]:
def are_words_in_string(word_list, input_string):
    for word in word_list:
        if word in input_string:
            return True
    return False

In [17]:
query = "treasure"
query_token = textprepo.preprocess_text(query)

def filter_fn(metadata):
    return any(word in query for word in list(query_token)) or metadata["score"] > 5.0


results = new_db.similarity_search(query, filter=filter_fn, k=5)
results

[Document(page_content="what follows is a mad dash by not only the treasure hunters but also the inhabitants of koumi to secure anne and mary's long lost booty. all the while, however, conan, kogorou, and the police search for the one thing far greater than riches—justice.", metadata={'anime_id': 2171, 'cast': 'UNKNOWN', 'episodes': '1.0', 'genre': 'Adventure, Comedy, Mystery', 'source': 'Manga', 'Duration': '1 hr 45 min', 'name': 'Detective Conan Movie 11: Jolly Roger in the Deep Azure', 'tokens': 'treasure pay trip chaos detective dare mad second conan send mystery big thing reef mouri robbery manga regard boy harsh azure acquaint famous hill encounter long reason booty daughter group \n\n discover cutlass far edogawa read sonoko roger fever anne scenic search follow movie hunt justice entertainment museum visit tourist plunge small isle mix murder jolly secure wonder join legend mary kogorou arrive lose scour friend recent clue pistol 11 water tourism deep rich bang luscious coral r

In [10]:
context = list(set([results[x].page_content for x in range(len(results))]))

In [16]:
results[0].metadata

{'anime_id': 2171,
 'cast': 'UNKNOWN',
 'episodes': '1.0',
 'genre': 'Adventure, Comedy, Mystery',
 'source': 'Manga',
 'Duration': '1 hr 45 min',
 'name': 'Detective Conan Movie 11: Jolly Roger in the Deep Azure',
 'tokens': 'treasure pay trip chaos detective dare mad second conan send mystery big thing reef mouri robbery manga regard boy harsh azure acquaint famous hill encounter long reason booty daughter group \n\n discover cutlass far edogawa read sonoko roger fever anne scenic search follow movie hunt justice entertainment museum visit tourist plunge small isle mix murder jolly secure wonder join legend mary kogorou arrive lose scour friend recent clue pistol 11 water tourism deep rich bang luscious coral room island soon inhabitant good suzuki come bonnie adventure pirate hotel dash hunter great shore duo run comedy koumi house unknown police tms',
 'score': 7.43,
 'producer': 'Big Bang',
 'studio': 'TMS Entertainment',
 'licensors': 'UNKNOWN'}

In [11]:
anime_id_list = set([results[x].metadata['anime_id'] for x in range(len(results))])

In [15]:
mistral = models.MistralInstruct("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/prompting/mistral-7b-instruct-v0.2-gguf/mistral-7b-instruct-v0.2.Q5_K_M.gguf", n_ctx=5000, n_gpu_layers=-1)

In [20]:
@guidance
def anime_recommend(lm, content, model):
    model.echo = False
    prompt = "Summarize in a full sentence the following text."
    lm = model + f"{prompt}: {content}\n"
    lm += "Summary: " + gen(
        name="summary",
        temperature=0.5,
        max_tokens=50,
    )
    return lm


@guidance
def anime_summarize(lm, content, model):
    model.echo = False
    prompt = "Summarize the following and descriptions of the 5 anime into one paragraph"
    lm = model + f"{prompt}: {content}"
    lm += "Summary: " + gen(
        name="full_summary",
        temperature=0,
        max_tokens=200,
    )
    return lm


def ani_recs(question: str) -> str:
    output = new_db.similarity_search(question, filter=filter_fn, k=5)
    rec_list = []
    for x in tqdm(range(len(output))):
        lm = mistral + anime_recommend(output[x].page_content, mistral)
        rec_list.append(
            {
                "anime_name": output[x].metadata['name'],
                "Description": lm["summary"],
                "anime_id": output[x].metadata["anime_id"],
            }
        )
    summary = " ".join([x["Description"] for x in rec_list])
    lm = mistral + anime_summarize(summary, mistral)
    fstring = f"""
    Summary: {lm['full_summary']}
    Dishes:
    1. {rec_list[0]['anime_name']}
    2. {rec_list[1]['anime_name']}
    3. {rec_list[2]['anime_name']}
    4. {rec_list[3]['anime_name']}
    5. {rec_list[4]['anime_name']}
    """
    return fstring

In [21]:
recs = ani_recs("I like pirates")

  0%|          | 0/5 [00:00<?, ?it/s]